In [2]:
import pandas as pd
import numpy as np
import os

In [6]:
df = pd.read_csv('../data/donnees-hospitalieres-covid19-2020-05-05-19h00.csv', sep=';')

In [7]:
df

,dep,sexe,jour,hosp,rea,rad,dc
0,01,0,2020-03-18,2,0,1,0
1,01,1,2020-03-18,1,0,1,0
2,01,2,2020-03-18,1,0,0,0
3,02,0,2020-03-18,41,10,18,11
4,02,1,2020-03-18,19,4,11,6
...,...,...,...,...,...,...,...
14845,974,1,2020-05-05,4,2,54,0
14846,974,2,2020-05-05,1,0,58,0
14847,976,0,2020-05-05,44,6,105,11
14848,976,1,2020-05-05,28,5,54,8
